In [1]:
#Import libraries
import numpy as np
import scipy as sp
from scipy import signal
import math
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
#Input the test number (ref: Test Records)

#Test number on the record
test_number = 4
test_id = 'Foam' #Test identifier
channel = 2 #Choose scope channel (for different sensitivity)

In [3]:
#Identify test from records
if test_id == 'NAM':
    test_record = pd.read_excel(r'F:\My Drive\Oxford 2022-23\NAM 3-4\NAME Test Records.xlsx')
elif test_id == 'Foam':
    test_record = pd.read_excel(r'F:\My Drive\Oxford 2022-23\DIHPB_Foam\Foam Test Records.xlsx')
    
record = pd.DataFrame(test_record, columns = ['Test No.','Sample Name','Thickness Avg','Firing Pressure','Impact Velocity','Projectile Mass','Save File'])
t_number = record['Test No.']
name = record['Sample Name']
vel = record['Impact Velocity']
thickness = record['Thickness Avg']
mass = record['Projectile Mass']
scope = record['Save File']

#Locator
m = list(t_number).index(test_number)

#Sample Name
test_name = name[m]
print(test_name)
#Scope data file name
scope_data = scope[m]

#File directory
if test_id =='NAM':
    up_directory = r'F:\My Drive\Oxford 2022-23\NAM 3-4\Test Data'
elif test_id =='Foam':
    up_directory = r'F:\My Drive\Oxford 2022-23\DIHPB_Foam\Test Data'
raw_data = os.path.join(up_directory,scope_data+'.'+'csv')
data = pd.read_csv(raw_data,skiprows=14)


#Create dataframe
if channel == 1:
    df_CH1 = pd.DataFrame(data, columns = ['TIME','CH1'])
    time = df_CH1['TIME'] #assign time column
    time = time.dropna()
    v1 = df_CH1['CH1'] #assign ch1 signal
    v1 = v1.dropna()
    v1 = v1-v1[0]
elif channel ==2:
    df_CH1 = pd.DataFrame(data, columns = ['TIME','CH2'])
    time = df_CH1['TIME'] #assign time column
    time = time.dropna()
    v1 = df_CH1['CH2'] #assign ch1 signal
    v1 = v1.dropna()
    v1 = v1-v1[0]
#Save directory
if test_id == 'NAM':
    save_dir = r'F:\My Drive\Oxford 2022-23\NAM 3-4\Analysed_Python'
elif test_id == 'Foam':
    save_dir = r'F:\My Drive\Oxford 2022-23\DIHPB_Foam\Analysed_Python'
analysed_data = os.path.join(save_dir,test_name)
if not os.path.exists(analysed_data):
    os.makedirs(analysed_data)

Rohacell_Test_3


In [4]:
#Test Parameters - *not to be changed unless re-calibrated*

b = 2/6; #wheatstone bridge
amp = 500; #amplifier
g = 1/2.1; #gauge
E_b = 210e9; #elastic modulus of bar (Steel)
r_b = (19.925/2)*1e-3; #radius of bar (m)
r_p = (11.91/2)*1e-3; #projectile radius (m)
r_a = (25.35/2)*1e-3; #anvil radius (m)
vel_p = vel[m]; #projectile velocity (m/s)
d_s = thickness[m]*1e-3; #sample thickness (m)
m_p = mass[m]*1e-3; #projectile mass (kg)
m_a = 239.4e-3  #anvil mass (kg)

In [5]:
#Isolate the signal (ie the first pulse)
%matplotlib 
plt.figure(1)
plt.plot(time,v1)
#Click before and after the initial pulse to zoom in
x1 = plt.ginput(2) 
plt.close()
#We use time as the reference
t_1 =x1[0][0]
t_2 = x1[1][0]
#locate indices in the time array
i=0
while time[i]<t_1:
    i=i+1
A=i    
j=0
while time[j]<t_2:
    j=j+1
B=j
#Isolated pulse for both signal and time
zoom_v = v1[A:B]
zoom_v = zoom_v.reset_index(drop=True)
zoom_time = time[A:B]
zoom_time = zoom_time.reset_index(drop=True)

#Click for beginning and end of the pulse
plt.figure(2)
plt.plot(zoom_time,zoom_v)
x2 =plt.ginput(2)
plt.close()
t_begin =x2[0][0]
t_end =x2[1][0]
#locate indices in the time array
p=0
while zoom_time[p]<t_begin:
    p=p+1    
q=0
while zoom_time[q]<t_end:
    q=q+1
#Isolated pulse for both signal and time
new_v = zoom_v[p:q]
new_v = new_v.reset_index(drop=True)
new_time = zoom_time[p:q]
new_time = new_time.reset_index(drop=True)


Using matplotlib backend: <object object at 0x000001F6EEED2F00>


In [10]:
#Parameter Calculations
strain = b*g*new_v/amp #calibration parameters
force = (np.pi*r_b**2)*E_b*strain #force on bar (E = stress/strain)

#Projectile decceleration
if test_id == 'NAM':
    stress_p = -force/(np.pi*r_p**2) #stress on projectile (assume N3)
    acc = force/m_p #from F = ma

elif test_id == 'Foam':
    stress_p = -force/(np.pi*r_b**2) #stress on anvil (assume N3)
    acc = force/m_a 

#Projectile velocity and displacement
velocity = []
displacement = []
#initial conditions
velocity.append(vel_p) 
displacement.append(0)
dt = (new_time[1]-new_time[0])

for k in range (1,len(acc)):  
    velocity.append(velocity[k-1] + acc[k-1]*dt)
    displacement.append(displacement[k-1] + velocity[k-1]*dt)
    

#Strain in sample
strain_s = []
for l in range(0,len(displacement)):
    eps = (displacement[l]-displacement[0])/d_s
    strain_s.append(eps)

#Strain Rate
srate = []
for i in range(0,len(strain_s)-1):
    eps_dot = (strain_s[i+1]-strain_s[i])/(new_time[i+1]-new_time[i])
    srate.append(eps_dot)


In [7]:
#Plots

%matplotlib 

fig, axs = plt.subplots(2,2)
plt.rcParams['axes.grid'] = True

#Force
axs[0,0].plot(new_time,-force, 'tab:red')
axs[0,0].set_ylabel('Force (N)')
axs[0,0].set_xlabel('Time (s)')

#Velocity
axs[0,1].plot(new_time,velocity, 'tab:blue')
axs[0,1].set_ylabel('Velocity ($ms^{-1}$)')
axs[0,1].set_xlabel('Time (s)')

#Displacement
axs[1,0].plot(new_time,displacement, 'tab:orange')
axs[1,0].set_ylabel('Displacement (m)')
axs[1,0].set_xlabel('Time (s)')

#Strain
axs[1,1].plot(new_time,strain_s, 'tab:green')
axs[1,1].set_ylabel('Strain')
axs[1,1].set_xlabel('Time (s)')

fig.tight_layout()

#Save Figure
filename = analysed_data + '\Parameters' + '.png'
plt.savefig(filename)

#Stress-Strain Curve
plt.figure(4)
plt.plot(strain_s,stress_p*1e-6)
plt.ylabel('Stress (MPa)')
plt.xlabel('Strain')
plt.title('Stress-Strain')
plt.grid(True)

#Save Figure
filename2 = analysed_data + '\Stress-Strain' + '.png'
plt.savefig(filename2)


Using matplotlib backend: TkAgg


In [8]:
#Save dataset as csv file
#Test data
raw_dataset = pd.DataFrame({'Raw Time': time, 'Voltage': v1}) 
#Analysed data
analysed_dataset = pd.DataFrame({'Time': new_time, 'Displacement': displacement, 'Velocity': velocity, 'Acceleration': acc, 'Strain': strain_s, 'Stress': stress_p})
#Combined data
all_dataset = pd.concat([raw_dataset, analysed_dataset], axis=1)

#Convert and save as csv
filename3 = analysed_data + '\\' + test_name + '.csv'
all_dataset.to_csv(filename3, index=False)

#For combined plot
if test_id == 'NAM':
    filename4 = os.path.join(r'F:\My Drive\Oxford 2022-23\NAM 3-4\Figures\Planar\NAM 4\Sample',test_name + '.csv')
    all_dataset.to_csv(filename4, index=False)
elif test_id == 'Foam':
    filename4 = os.path.join(r'F:\My Drive\Oxford 2022-23\DIHPB_Foam\Figures',test_name + '.csv')
    all_dataset.to_csv(filename4, index=False)

In [16]:
if test_id =='Foam':
    fig,ax1 =plt.subplots()
    ax1.plot(new_time[:-1],srate)
    ax1.set_xlabel('time (s)')
    ax1.set_ylabel('strain rate (/s)')
    ax1.tick_params('y')
    
    ax2 = ax1.twinx()
    ax2.plot(new_time[:-1],strain_s[:-1])
    ax2.set_ylabel('strain')
    ax2.tick_params('y')
    
    
    plt.show()